# test DL models

## so much import

In [1]:
import os
import sys
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [106]:
from importlib import reload

In [12]:
top_path = os.path.dirname(os.path.abspath("./"))
sys.path.append(top_path)
print(top_path)

/home/wergillius/Project/UTR_VAE


In [108]:
import utils
reload(utils)
reload(DL_models)

/home/wergillius/Project/UTR_VAE


<module 'model.DL_models' from '/home/wergillius/Project/UTR_VAE/model/DL_models.py'>

In [21]:
from model import DL_models
from model import reader

### read data

In [23]:
# define data
dataset = reader.UTR_dataset(cell_line='A549')

train_loader,val_loader,test_loader = reader.get_splited_dataloader(dataset,
                                                                    ratio=[0.7,0.2,0.1],
                                                                    batch_size=10,
                                                                    num_workers=4)

In [27]:
train_iter=iter(train_loader)

In [101]:
train_loader.batch_size

10

In [102]:
len(train_loader)

278

In [28]:
# from train loader get some data
batch_data = next(train_iter)

X, y = batch_data
X = X.float()
y = y.long()

In [29]:
X.shape , y.shape

(torch.Size([10, 100, 4]), torch.Size([10]))

## Test DL models

In [32]:
# define some parameters for LSTM

laten_dim = 32
hidden_size = 8
out_dim = 2*hidden_size*X.shape[1]

In [31]:
L_encoder = DL_models.LSTM_backbond(input_size=4,
                                    hidden_size=8,
                                    num_layers=3,
                                    bidirectional=True)

In [34]:
L_decoder = DL_models.LSTM_backbond(input_size=laten_dim,
                                    hidden_size=4,
                                    num_layers=3,
                                    bidirectional=False)

In [35]:
model = DL_models.VAE(L_encoder,L_decoder,latent_dim=laten_dim,out_dim=out_dim)

In [122]:
import logging

In [124]:
logger = utils.setup_logs(os.path.join(utils.script_dir,'script'),'testing')

FileNotFoundError: [Errno 2] No such file or directory: '/home/wergillius/UTR_VAE/script/testing.log'